## Project Name
> Auxilary tools and solution for pre/post
processing of audio and video based on ffmpeg
inspired by [ffmpeg]('https://pypi.org/project/python-ffmpeg-video-streaming/)

# 1. Installing/updates of libraries

In [1]:

# from datetime import datetime
!pip install python-ffmpeg-video-streaming
!apt install ffmpeg
!pip install ffmpy

from IPython.display import HTML
from base64 import b64encode

import ffmpeg_streaming
from ffmpeg_streaming import Formats
import ffmpy
import os
import datetime as dt
import shutil



Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
  Created wheel for ffmpy: filename=ffmpy-0.2.3-cp36-none-any.whl size=4623 sha256=fe3c0f29f7b8d7ef6a674a9ae19e194112666b3145c73aa9c506040a3def20cd
  Stored in directory: /root/.cache/pip/wheels/99/51/33/ca9d92c9fa38a196ad8647f5fcc9eadbca8a96bd5f8a2b5f7d
Successfully built ffmpy


# 2. Mounting google drive

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Standard snipet to prevent random disconnects

This cell runs JS code to automatic reconnect to runtime.

In [3]:
import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("Done.")

<IPython.core.display.Javascript object>

Done.


## Check GPU

*   Google Colab can provide you with one of Tesla graphics cards: K80, T4, P4 or P100
*   Here you can check the model of GPU before using some advanced features

In [ ]:
!nvidia-smi

Mon Dec 28 19:28:14 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Creating/checking if exist temporary folder '/content/drive/MyDrive/ff_temp/' for pre/post processing

# Useful functions

In [ ]:
def folder_check(path):
    try_num = 1
    oripath = path[:-1] if path.endswith('/') else path
    if(not os.path.exists(path)):
      os.mkdir(path)
      print(path, '   created')
    else:    
      while os.path.exists(path):
          print("Delete content of  existing folder " + path + "?(Y/N)")
          decision = input()
          if decision == "Y":
              shutil.rmtree(path, ignore_errors=True)
              os.mkdir(path)

              break
          else:
            path = oripath + "_%d/"%try_num
            try_num += 1
            if(not os.path.exists(path)):
              print("Create new folder " + path + "?(Y/N)")
              decision = input()
              if decision == "Y":
                os.mkdir(path)
                print(path, 'was created')
                break
      return path

In [ ]:
# generating file name with timestamp info for later comparison
def gen_file_with_timestamp(name='name'):
    timestamp = dt.datetime.now()
    ts1 = timestamp.strftime("%A%d%B%Y%m")
    ts2 = timestamp.strftime("%H")
    ts3 = timestamp.strftime("%M")
    ts4 = timestamp.strftime("%S")
    file_name= name + ts1 +"-"+ ts2+"-"+ ts3+"-"+ ts4+".mp4"
    return file_name

In [ ]:
filename = gen_file_with_timestamp(name='name')
filename

'nameTuesday29December202012-15-08-59.mp4'

In [ ]:
def save_m3u8(path,  time_chunk):
  print('chunks will be saved in {} directory to stop press stop button on the left'.format(path) )

  os.system(f"ffmpeg -re -i https://tagesschau-lh.akamaihd.net/i/tagesschau_1@119231/master.m3u8 -vcodec libx264 -vb 500000 -g 60 -vprofile main -acodec aac -ab 128000 -ar 48000 -ac 2 -vbsf h264_mp4toannexb -b 1000k -minrate 1000k -maxrate 1000k -strict experimental -f stream_segment -segment_format mpegts -segment_time {time_chunk} -segment_atclocktime 1 -reset_timestamps 1 -strftime 1 {path}%H%M%S.mp4")

# 3. Playing video file in colab window

In [ ]:
video_file='/content/drive/MyDrive/ff_temp/204800recod.mp4'
mp4 = open(video_file,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

Output hidden; open in https://colab.research.google.com to view.



> Preparing temporary folder for video processing



In [ ]:
path = folder_check('/content/drive/MyDrive/ff_temp/')

Delete content of  existing folder /content/drive/MyDrive/ff_temp/?(Y/N)
N
Delete content of  existing folder /content/drive/MyDrive/ff_temp_1/?(Y/N)
Y


In [ ]:
os.system(f"ffmpeg -i {input_video} -vcodec libx264 {output_video}")

0



> 3a. Decoding once again mp4  If it gives error there is need to decode pice of mp4 again like below



In [4]:
input_video='/content/drive/MyDrive/ff_temp/205101.mp4'
output_video='/content/drive/MyDrive/ff_temp/205101recod.mp4'

In [ ]:
path

'/content/drive/MyDrive/ff_temp_1/'

As a source of video for later checking  saved m3u8 video stream will be used

In [ ]:
time_chunk = 210 # in seconds
save_m3u8(path,  time_chunk)

chunks will be saved in /content/drive/MyDrive/ff_temp_2/ directory to stop press stop button on the left


In [ ]:
path = '/content/drive/MyDrive/ff_temp/'

In [ ]:
def recode_mp4_from_folder(path):
  to_recode = [path+s  for s in os.listdir(path) if s.endswith('.mp4')]

  #creating list of tuples source and result name
  to_recode_out=[]
  for i in to_recode:
    to_recode_out.append((i, i[:-4]+'recode'+i[-4:]))
    
  #recoding saved chunks
  for  j in to_recode_out:
    print(j[0], ' --> vcodec libx264 --> ',j[1] )
    os.system(f"ffmpeg -i {j[0]} -vcodec libx264 {j[1]}")
    os.remove(j[0])
    print(j[0], ' --> was removed after recoding ')

In [ ]:
recode_mp4_from_folder(path)

/content/drive/MyDrive/ff_temp/200001.mp4  --> vcodec libx264 -->  /content/drive/MyDrive/ff_temp/200001recode.mp4
/content/drive/MyDrive/ff_temp/200001.mp4  --> was removed after recoding 
/content/drive/MyDrive/ff_temp/201500.mp4  --> vcodec libx264 -->  /content/drive/MyDrive/ff_temp/201500recode.mp4
/content/drive/MyDrive/ff_temp/201500.mp4  --> was removed after recoding 
/content/drive/MyDrive/ff_temp/204800.mp4  --> vcodec libx264 -->  /content/drive/MyDrive/ff_temp/204800recode.mp4
/content/drive/MyDrive/ff_temp/204800.mp4  --> was removed after recoding 
/content/drive/MyDrive/ff_temp/205101.mp4  --> vcodec libx264 -->  /content/drive/MyDrive/ff_temp/205101recode.mp4
/content/drive/MyDrive/ff_temp/205101.mp4  --> was removed after recoding 


#4. Cutting with reencoding 

In [ ]:
#start_end is defined as tuple start_end=('0:01', '0:30')

def trim_mp4(input_video, start_end):

  output_video= input_video[:-4]+'_trimed_'+start_end[0][-2:]+'_to_'+start_end[1][-2:]+input_video[-4:]
  print('trimed video will be  saved to {} ...'.format(output_video))
  os.system(f"ffmpeg -i {input_video} -ss {start_end[0]} -to {start_end[1]} -c:v libx264 -crf 30 {output_video}")
  return output_video

In [ ]:
start_end=('0:01', '0:43')
input_video='/content/drive/MyDrive/ff_temp/200001recode.mp4'
to_play=trim_mp4(input_video, start_end)

trimed video will be  saved to /content/drive/MyDrive/ff_temp/200001recode_trimed_01_to_43.mp4 ...


In [ ]:
video_file=to_play
mp4 = open(video_file,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

Output hidden; open in https://colab.research.google.com to view.